In [ ]:
import pandas as pd

In [ ]:
data= "Baselines/DeepCDR-master/data/CCLE/genomic_mutation_34673_demap_features.csv"
df = pd.read_csv(data, index_col=False)
df.shape

In [ ]:
cell_name=list(df.iloc[:,0])

cell_name


In [ ]:
print(df.columns)

In [ ]:
import requests

def get_drug_smiles(drug_id):
    """
    Retrieve the SMILES string for a given drug ID using the PubChem API.
    """
    # Construct the API URL
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{drug_id}/property/CanonicalSMILES/json"

    # Send the API request
    response = requests.get(url)

    # Parse the JSON response
    json_data = response.json()

    # Extract the SMILES string
    smiles = json_data["PropertyTable"]["Properties"][0]["CanonicalSMILES"]

    return smiles


In [ ]:
smiles = get_pubchem_smiles(9826528)
print(smiles)


In [ ]:

import requests
import time


In [ ]:
def execute(name):

    drug_names = []
    drug_names.append(name)

    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

    url_for_synonyms_to_cid = f"{base_url}/compound/name/synonyms/json"
    response_pub_chem_synonyms = requests.post(url_for_synonyms_to_cid, {"name": name})
    if response_pub_chem_synonyms.status_code > 202:
        raise Exception(f"PUBChem ({url_for_synonyms_to_cid})\nResponse code {response_pub_chem_synonyms.status_code}\n"
                        f"{response_pub_chem_synonyms.text}")
    response_pub_chem_synonyms = response_pub_chem_synonyms.json()
    cid_record = response_pub_chem_synonyms["InformationList"]["Information"][0]

    time.sleep(1)

    compound_properties = {}
    url_for_pub_chem_smiles_iupac = f"{base_url}/compound/cid/property/IUPACName,CanonicalSMILES/json"
    response_pub_chem_smiles_iupac = requests.post(url_for_pub_chem_smiles_iupac, {"cid": cid_record["CID"]})
    if response_pub_chem_smiles_iupac.status_code > 202:
        raise Exception(f"PUBChem ({url_for_pub_chem_smiles_iupac})\n"
                        f"Response code {response_pub_chem_smiles_iupac.status_code}\n"
                        f"{response_pub_chem_smiles_iupac.text}")
    response_pub_chem_smiles_iupac = response_pub_chem_smiles_iupac.json()
    for params in response_pub_chem_smiles_iupac["PropertyTable"]["Properties"]:
        try:
            compound_properties[params["CID"]] = [params["CanonicalSMILES"], params["IUPACName"],
                                                  cid_record["Synonym"][0]]
        except KeyError:
            compound_properties[params["CID"]] = ["", "", ""]
    SMILES= list(compound_properties.values())[0][0]
    return SMILES

In [ ]:
a=execute("9826528")
a

In [ ]:
print(a)

In [ ]:
import numpy as np
import pandas as pd

pd.read_excel("data/CCLE/drug_id_name.xlsx", header=None)

In [ ]:
df=pd.read_csv("data/CCLE/drugResponse_ccle.csv", index_col=0)
df

In [ ]:
tmp=[]
for idx,row in df.iterrows():
  for col in df.columns:
    tmp.append((col,idx,df.loc[idx,col]))

In [ ]:
print(tmp[5])

In [5]:
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GraphConv, TopKPooling
from torch_geometric.nn import global_max_pool as gmp
from torch_geometric.nn import global_mean_pool as gap

path = 'PROTEINS'
dataset = TUDataset(path, name='PROTEINS')
dataset = dataset.shuffle()
n = len(dataset) // 10
test_dataset = dataset[:n]
train_dataset = dataset[n:]
test_loader = DataLoader(test_dataset, batch_size=60)
train_loader = DataLoader(train_dataset, batch_size=60)


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = GraphConv(dataset.num_features, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)

        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, dataset.num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.log_softmax(self.lin3(x), dim=-1)

        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


def train(epoch):
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        print("target shape:", data.y.shape)
        print("output shape:",output.shape)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        pred = model(data).max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)


for epoch in range(1, 201):
    loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.5f}, Train Acc: {train_acc:.5f}, '
          f'Test Acc: {test_acc:.5f}')

target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size([60])
output shape: torch.Size([60, 2])
target shape: torch.Size(

KeyboardInterrupt: 

In [6]:
import torch
from torch.utils.data import DataLoader,Dataset
from utils import MyTrainDataset

ImportError: cannot import name 'MyTrainDataset' from 'utils' (C:\Users\molou\miniconda3\lib\site-packages\utils\__init__.py)

In [4]:
pip install utils

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simpleNote: you may need to restart the kernel to use updated packages.

